In [1]:
from datasets import load_dataset
import random
import torch

In [2]:
from tqdm.notebook import tqdm

In [3]:
q_a_train = {}
q_a_test = {}

In [4]:
# Squad v1

In [5]:
splits = ["train", "validation"]
dicts = {
    "train":q_a_train,
    "validation":q_a_test
}
ds = load_dataset("squad")
print(ds)

Found cached dataset squad (/home/amanrai/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [6]:
for split in splits:
    for row in tqdm(ds[split]):
        q_a = dicts[split]
        if (row["context"] not in q_a):
            q_a[row["context"]] = {}
            q_a[row["context"]]["questions"] = []
            q_a[row["context"]]["answers"] = []
            q_a[row["context"]]["style"] = "ClosedQA"
        q_a[row["context"]]["questions"].append(row["question"])
        q_a[row["context"]]["answers"].append(row["answers"]["text"])

  0%|          | 0/87599 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

In [7]:
list(q_a.keys())[:10]

['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced

In [8]:
ds = load_dataset("squad_v2")

Found cached dataset squad_v2 (/home/amanrai/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
for split in splits:
    for row in tqdm(ds[split]):
        q_a = dicts[split]
        if (row["context"] not in q_a):
            q_a[row["context"]] = {}
            q_a[row["context"]]["questions"] = []
            q_a[row["context"]]["answers"] = []
            q_a[row["context"]]["style"] = "ClosedQA"
        q_a[row["context"]]["questions"].append(row["question"])
        q_a[row["context"]]["answers"].append(row["answers"]["text"])

  0%|          | 0/130319 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

In [10]:
ds = load_dataset("sciq")

Found cached dataset sciq (/home/amanrai/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
for split in splits:
    for row in tqdm(ds[split]):
        q_a = dicts[split]
        if (row["support"] not in q_a):
            q_a[row["support"]] = {}
            q_a[row["support"]]["questions"] = []
            q_a[row["support"]]["answers"] = []
            q_a[row["support"]]["style"] = "ClosedQA"
        q_a[row["support"]]["questions"].append(row["question"])
        q_a[row["support"]]["answers"].append(row["correct_answer"])

  0%|          | 0/11679 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
from transformers import AutoTokenizer

In [13]:
stab = AutoTokenizer.from_pretrained("StabilityAI/stablelm-base-alpha-3b")

In [14]:
import numpy as np

In [15]:
def get_lengths(tokenizer):
    _lengths = []
    for split in splits:
        q_a = dicts[split]
        for key in tqdm(q_a):
            for i in range(len(q_a[key]["questions"])):
                q = q_a[key]["questions"][i]
                a = q_a[key]["answers"][i]
                _t = f"T T {key} T {q} T {a} T"
                _lengths.append(len(tokenizer.encode(_t)))
        return _lengths

In [16]:
tokenizers = [stab]
for tokenizer in tokenizers:
    _lengths = get_lengths(tokenizer)
    print(tokenizer)
    print("Max Length:", np.max(_lengths))

  0%|          | 0/29504 [00:00<?, ?it/s]

GPTNeoXTokenizerFast(name_or_path='StabilityAI/stablelm-base-alpha-3b', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)
Max Length: 1198


In [17]:
vector_lists = {}

for key in dicts:
    vector_lists[key] = []

In [18]:
import requests
from tqdm.notebook import trange
import json

In [19]:
%%time
for split in dicts:
    vectors = vector_lists[split]
    keys = list(dicts[split].keys())
    for i in trange(len(keys)):
        vector = requests.post("http://localhost:8200/embed", json={"chunk":keys[i]}).json()
        vector = json.loads(vector)[0]
        vectors.append(vector)

  0%|          | 0/29504 [00:00<?, ?it/s]

  0%|          | 0/2955 [00:00<?, ?it/s]

CPU times: user 1min 14s, sys: 7.43 s, total: 1min 21s
Wall time: 6min 37s


In [20]:
cosine_sim = torch.nn.CosineSimilarity()
unmatched_pairs = {}
for split in splits:
    unmatched_pairs[split] = []
    vectors = vector_lists[split]
    _vectors = torch.FloatTensor(np.asarray(vectors)).cuda()
    for i in trange(len(vectors)):
        vector = vectors[i]
        _v = torch.FloatTensor([vector]).cuda()
        _sim = cosine_sim(_v, _vectors)
        sorted, indices = torch.sort(_sim)
        next_highest = indices.cpu().numpy()[-2]
        unmatched_pairs[split].append((i, next_highest))

#     print(keys[i],"\n\n", keys[next_highest])
#     break

  0%|          | 0/29504 [00:00<?, ?it/s]

  0%|          | 0/2955 [00:00<?, ?it/s]

In [21]:
user_handle = "<|USER|>"
system_handle = "<|SYSTEM|>"
bot_handle = "<|ASSISTANT|>"
instruction = "<|INSTRUCTION|>"
context_token = "<|CONTEXT|>"
answer_not_in_context_token = "<|ANSWER_NOT_IN_CONTEXT|>"
answer_in_context_token = "<|ANSWER_IN_CONTEXT|>"
eos_token = "</s>"


possible = "{context_token} \n {context} \n {user_handle} {question} \n {bot_handle} {answer_in_context_token} {response}{eos_token}"
not_possible = "{context_token} \n {context} \n {user_handle} {question} \n {bot_handle} {answer_not_in_context_token} Not answerable. {eos_token}"

In [22]:
f_ds = {}
for split in splits:
    f_ds[split] = []

In [ ]:
#first add the un-answerable questions

In [23]:
split = "train"
c_, q_ = unmatched_pairs[split][0]
print(list(dicts[split].keys())[c_])
print(list(dicts[split].keys())[q_])

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Because of its Catholic identity, a number of religious buildings stand on campus. The Old College building has become one of two seminaries on campus run by the Congregation of Holy Cross. The current Basilica of the Sacred Heart is located on the spot of Fr. Sorin's original church, which became too s

In [24]:
for split in splits:
    q_a = dicts[split]
    keys = list(q_a.keys())
    for i in trange(len(unmatched_pairs[split])):
        pair = unmatched_pairs[split][i]
        _a = q_a[keys[pair[1]]]["questions"]
        _context = keys[pair[0]]
        _question = random.choice(_a)
        _q = not_possible.format(context_token=context_token, context=_context, user_handle=user_handle, question=_question, bot_handle=bot_handle, answer_not_in_context_token =answer_not_in_context_token, eos_token=eos_token, answer_in_context_token=answer_in_context_token)
        f_ds[split].append(_q)

  0%|          | 0/29504 [00:00<?, ?it/s]

  0%|          | 0/2955 [00:00<?, ?it/s]

In [25]:
print(f_ds["train"][:10])

['<|CONTEXT|> \n Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. \n <|USER|> In which architectural style is the Basilica of the Sacred Heart at Notre Dame made? \n <|ASSISTANT|> <|ANSWER_NOT_IN_CONTEXT|> Not answerable. </s>', "<|CONTEXT|> \n As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-r

In [26]:
possible = "{context_token} \n {context} \n {user_handle} {question} \n {bot_handle} {answer_in_context_token} {response} {eos_token}"

In [35]:
for split in dicts:
    _dict = dicts[split]
    for key in tqdm(_dict):
        for i in range(len(_dict[key]["questions"])):
            _q = _dict[key]["questions"][i]
            try:
                _a = _dict[key]["answers"][i]
                if (type(_a) == list):
                    _a = _a[0]
                _a = _a[0].upper() + _a[1:]
                _f = possible.format(context_token=context_token, context=key, user_handle=user_handle, question=_q, bot_handle=bot_handle, response = _a, eos_token=eos_token, answer_in_context_token=answer_in_context_token)
                f_ds[split].append(_f)
            except:
#                 print(_q, _dict[key]["answers"])
                pass
            

  0%|          | 0/29504 [00:00<?, ?it/s]

  0%|          | 0/2955 [00:00<?, ?it/s]

In [36]:
len(f_ds["validation"])

40109

In [37]:
print(f_ds["train"][-10:])

['<|CONTEXT|> \n Mollusks have a hard outer shell. There is a layer of tissue called the mantle between the shell and the body. \n <|USER|> What is the layer of tissue between the body and shell called? \n <|ASSISTANT|> Mantle </s>', '<|CONTEXT|> \n Roots are covered with thin-walled dermal cells and tiny root hairs. These features are well suited to absorb water and dissolved minerals from the soil. \n <|USER|> Well suited to absorb water and dissolved minerals from the soil, thin-walled dermal cells and tiny hairs cover what basic plant structures? \n <|ASSISTANT|> Roots </s>', '<|CONTEXT|> \n The Heart The heart is a complex muscle that consists of two pumps: one that pumps blood through pulmonary circulation to the lungs, and the other that pumps blood through systemic circulation to the rest of the body’s tissues (and the heart itself). The heart is asymmetrical, with the left side being larger than the right side, correlating with the different sizes of the pulmonary and systemic

In [29]:
np.random.shuffle(f_ds["train"])
np.random.shuffle(f_ds["validation"])

f_ds["train"].extend(f_ds["validation"][int(0.25*len(f_ds["validation"])):])
f_ds["validation"] = f_ds["validation"][:int(0.25*len(f_ds["validation"]))]

In [30]:
print(len(f_ds["train"]))

230943


In [31]:
print(len(f_ds["validation"]))

5113


In [32]:
for i in trange(100):
    print(random.choice(f_ds["train"]))

  0%|          | 0/100 [00:00<?, ?it/s]

<|CONTEXT|> 
 Like all organisms, plants detect and respond to stimuli in their environment. Their main response is to change how they grow. 
 <|USER|> What do plants respond to in their environment? 
 <|ASSISTANT|> Changes in sunlight or water </s>
<|CONTEXT|> 
 When used with a load that has a torque curve that increases with speed, the motor will operate at the speed where the torque developed by the motor is equal to the load torque. Reducing the load will cause the motor to speed up, and increasing the load will cause the motor to slow down until the load and motor torque are equal. Operated in this manner, the slip losses are dissipated in the secondary resistors and can be very significant. The speed regulation and net efficiency is also very poor. 
 <|USER|> How efficient is such a design? 
 <|ASSISTANT|> Very poor </s>
<|CONTEXT|> 
 Large brushes are desired for a larger brush contact area to maximize motor output, but small brushes are desired for low mass to maximize the spe

In [ ]:
with open("./qa_basic_dataset.json", "w") as f:
    f.write(json.dumps(f_ds))

In [ ]:
"trivia_qa"

In [ ]:
new_datasets = ["trivia_qa", "b-mc2/wikihow_lists", "VMware/open-instruct-v1.1-oasst-dolly-hhrlhf", "theblackcat102/sharegpt-english", "search_qa"]

In [ ]:
trivia_qa = load_dataset("trivia_qa", "rc.web")

In [ ]:
search_qa = load_dataset("search_qa", "train_test_val")

In [ ]:
search_qa

In [ ]:
for row in search_qa["train"]:
    print(json.dumps(row, indent=4))
    break

In [ ]:
for row in trivia_qa["train"]:
    print(row["question"])
    print("*************")
    print(json.dumps(row["search_results"], indent=4))
    print("**************")
    print(row["answer"])
    print("\n\n\n\n")
    break

In [ ]:
with open("./qa_basic_dataset.json", "r") as f:
    ds = json.loads(f.read())

In [ ]:
ds.keys()